In [14]:
pip install selenium

  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached attrs-24.2.0-py3-none-any.whl.metadata (11 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   -------------------- ------------------- 5.0/9.7 MB 33.7 MB/s eta 0:00:01
   ---------------------------------------- 9.7/9.7 MB 28.8 MB/s eta 0:00:00
Using cached websocket_client-1.8.0-py3-none-any.whl (58 kB)
Using cached attrs-24.2.0-py3-none-any.whl (63 kB)
Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 0.58.0
    Uninstalling websocket-client-0.58.0:
      Successfully uninstalled websocket-client-0.58.0
  Attempting uninstall: attrs
    Found existing installation: attrs 23.1.0
    Uninstalling attrs-23.1.0:
      Successfully uninstalled attrs-23.1.0
Note: you may need to restart the kernel to use updated packages.


In [26]:
pip install webdriver_manager

Note: you may need to restart the kernel to use updated packages.


In [58]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException, TimeoutException, NoSuchElementException
import time
from selenium.webdriver.common.by import By

In [ ]:
def define_options(default_directory_path):   
    # Set Chrome options to download PDFs automatically instead of opening them
    options = Options()
    options.add_experimental_option('prefs', {
        'download.default_directory': default_directory_path,  # Path to save the files
        'download.prompt_for_download': False,  # Automatically download without asking
        'download.directory_upgrade': True,  # Automatically replace if file exists
        'plugins.always_open_pdf_externally': True,  # Disable PDF viewer
    })
    return options

def define_webdriver():
    # Set up the Selenium WebDriver with ChromeDriverManager
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=define_options(DEFAULT_DIRECTORY_PATH))
    
    # Open the quarterly reports page
    driver.get(url)
    
    driver.maximize_window()  # Maximize the window
    driver.implicitly_wait(10)  # Wait for page load
    return driver

def close_cookie_div(driver):
    try:
        cookie_div = driver.find_element(By.XPATH, "//div[@id='onetrust-banner-sdk']")
        if cookie_div:
            driver.find_element(By.XPATH, "//div[@id='onetrust-close-btn-container']/button").click()
    except NoSuchElementException:
        print("No cookie banner found or already closed.")

def find_and_download_shareholder_report(driver):
    try:
        # Find the link that contains 'Report to shareholders'
        pdf_links = driver.find_elements(By.XPATH, "//a[contains(text(),'Report to shareholders')]")

        for link in pdf_links:
            try:
                # Scroll into view to make sure the element is visible
                driver.execute_script("arguments[0].scrollIntoView();", link)

                # Explicitly wait for the element to be present and visible
                WebDriverWait(driver, 25).until(EC.visibility_of(link))

                # Attempt to click using JavaScript to bypass potential blockers
                driver.execute_script("arguments[0].click();", link)
                time.sleep(3)  # Wait for a few seconds to ensure the download starts

            except ElementClickInterceptedException:
                print("ElementClickInterceptedException: Trying to click using JavaScript.")
                # If intercepted, click using JavaScript to bypass obstruction
                driver.execute_script("arguments[0].click();", link)
                time.sleep(3)  # Wait for a few seconds to ensure the download starts
    except TimeoutException:
        print("TimeoutException: Element not found or clickable within the time limit.")
        driver.quit()
    except NoSuchElementException:
        print("NoSuchElementException: No 'Report to shareholders' link found.")

DEFAULT_DIRECTORY_PATH = 'C:\\Users\\MIS\\Lambton\\NLP\\Assignment-1'

# URL of the page to scrape
url = 'https://www.cibc.com/en/about-cibc/investor-relations/quarterly-results.html'

driver = define_webdriver()
close_cookie_div(driver)
find_and_download_shareholder_report(driver)

# Close the driver after downloading
driver.quit()


In [50]:
driver.quit()